In [ ]:
from DataPrepper import PrepData
from ChironModel import Chiron, SaveCB
from Utils import labelBaseMap, get_valid_taiyaki_filename, set_gpu_limit

set_gpu_limit(7000)

filename = get_valid_taiyaki_filename()

In [ ]:
prepData = PrepData(filename, RNN_LEN=300)
chiron = Chiron(prepData.get_max_label_len())
save_cb = SaveCB("models", "images", chiron.calculate_loss, prepData)

In [ ]:
for idx in range(prepData.get_len()):
    print(f"Epoch {idx}/{prepData.get_len()}")
#     try:
    a = next(prepData.train_gen())
    chiron.fit(a[0], a[1], initial_epoch=idx, epochs=idx+1, callbacks=[save_cb])
#     except Exception as e:
#         print(f"Error {e}, continuing...")